# Analisi del trend partendo da una serie intraday:

Aggreghiamo i risultati su base daily partendo dal caricamento del file del Gold Future a 60 minuti:

In [1]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__) # necessaria versione >= 1.9.0

import cufflinks as cf

# Per utilizzo con Notebooks
init_notebook_mode(connected=True)

# Per utilizzo offline
cf.go_offline()


filename = "@GC_60min.txt"
startDate = "20130101"
endDate = "20180810"

startDateParsed = datetime.date(int(startDate[0:4]), int(startDate[4:6]), int(startDate[6:]))
endDateParsed = datetime.date(int(endDate[0:4]), int(endDate[4:6]), int(endDate[6:]))

instrument = pd.read_csv(filename,
                         usecols=['Date','Time','Open','High','Low','Close','Up','Down'], 
                         parse_dates=[['Date', 'Time']])
print("Il file contiene", len(instrument), "record")
instrument.index = instrument['Date_Time']
instrument.drop('Date_Time',axis=1,inplace=True)
instrument['AvgPrice'] = round(instrument.iloc[:,0:4].mean(axis=1),2)
instrument['Range'] = round(instrument['High'] - instrument['Low'],2)
instrument['Body'] = abs(instrument['Open'] - instrument['Close'])
instrument['CloseOpenPerc'] = round((instrument['Close'] - instrument['Open']) / instrument['Open'] * 100,2)
instrument['BodyRangePerc'] = round(instrument['Body']/instrument['Range'] * 100,2)
# Percentuale della barra (all'interno del range) dove si attesta il close
instrument['CloseBarPerc'] = round((instrument['Close'] - instrument['Low']) * 100 / instrument['Range'],2)
instrument['DeltaClosePerc'] = round((instrument['Close'] - instrument['Close'].shift(1)) / instrument['Close'].shift(1) * 100,2)

instrument = instrument.loc[startDateParsed:endDateParsed]
#instrument.head(10)

# creazione Pandas DataFrame di lavoro con i soli campi utili
cluster = pd.DataFrame(index=instrument.Open.resample('D').first().dropna().index.copy())
cluster["Open"] = instrument.Open.resample('D').first().dropna()
cluster["Close"] = instrument.Close.resample('D').last().dropna()
cluster['DeltaClosePerc'] = round((cluster['Close'] - cluster['Close'].shift(1)) / cluster['Close'].shift(1) * 100,2)
cluster['CloseOpenPerc'] = round((cluster['Close'] - cluster['Open']) / cluster['Open'] * 100,2)

print(cluster.head(10))
print(cluster.tail(10))

cluster.drop("Open", axis=1, inplace=True)
cluster.drop("Close", axis=1, inplace=True)

4.7.1


Il file contiene 68540 record
              Open   Close  DeltaClosePerc  CloseOpenPerc
Date_Time                                                
2013-01-01  1748.6  1750.7             NaN           0.12
2013-01-02  1750.6  1759.0            0.47           0.48
2013-01-03  1759.0  1724.3           -1.97          -1.97
2013-01-04  1724.3  1728.5            0.24           0.24
2013-01-06  1728.5  1732.5            0.23           0.23
2013-01-07  1732.4  1721.2           -0.65          -0.65
2013-01-08  1721.1  1730.6            0.55           0.55
2013-01-09  1730.6  1729.1           -0.09          -0.09
2013-01-10  1729.0  1744.6            0.90           0.90
2013-01-11  1744.6  1734.3           -0.59          -0.59
              Open   Close  DeltaClosePerc  CloseOpenPerc
Date_Time                                                
2018-07-31  1229.8  1231.1            0.11           0.11
2018-08-01  1231.0  1227.6           -0.28          -0.28
2018-08-02  1227.6  1216.2           -0.93

In [2]:
print("")
# Grafichiamo la serie di variazioni percentuali
cluster['DeltaClosePerc'].iplot(kind='bar', color="blue", title="Multiperiod Trends")
cluster['CloseOpenPerc'].iplot(kind='bar', color="red", title="Intraperiod Trends")

# Stampa Plotly&Cufflinks delle distribuzioni Intraperiod e multiperiod
cluster['DeltaClosePerc'].iplot(kind='hist',
                                bins=300,
                                color="blue",
                                title="Distribution of Multiperiod Trends")

cluster['CloseOpenPerc'].iplot(kind='hist',
                               bins=300,
                               color="red",
                               title="Distribution of Intraperiod Trends")

cluster.iplot(kind='box', color=["red", "blue"], title="Distribution of Multiperiod & Intraperiod Trends")

print("")
print("GANDALF TREND ANALYZER - Powered by Gandalf Project R&D")
print("")
print("Correlation Matrix:")
print("")
print(cluster.corr())
print("")
print("Percentiles Plot:")
print("")
print(cluster.describe(percentiles=[.1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]))

print("")
print("-> UPTRENDS STATISTICS:")

# Intraperiod Analyzer
print("")
print("    -> INTRAPERIOD")
print("")
print("        Percentiles Benchmark:")
print("")
riferimentiPercentili = [.5, .6, .7, .8, .9]
for i in riferimentiPercentili:
    numberofIntraperiodPowerfulTrends = cluster[cluster.CloseOpenPerc >= cluster["CloseOpenPerc"].quantile(i)][
        "CloseOpenPerc"].count()
    IntraperiodPowerfulTrendsPerc = round(numberofIntraperiodPowerfulTrends / len(cluster) * 100, 2)
    if cluster["CloseOpenPerc"].quantile(i) >= 0:
        print("        Percentuale di Intraperiod Trend superiori o uguali al",
              i * 100, "° percentile ({} %) ->".format(round(cluster["CloseOpenPerc"].quantile(i), 2)),
              IntraperiodPowerfulTrendsPerc, "%")

print("")
print("        Static values Benchmark:")
print("")
riferimenti = [0, 0.5, 1., 1.5, 2.]
for i in riferimenti:
    numberofIntraperiodPowerfulTrends = cluster[cluster.CloseOpenPerc >= i]["CloseOpenPerc"].count()
    IntraperiodPowerfulTrendsPerc = round(numberofIntraperiodPowerfulTrends / len(cluster) * 100, 2)
    print("        Percentuale di Intraperiod Trend superiori o uguali a", i, "%", "->", IntraperiodPowerfulTrendsPerc, "%")

# Multiperiod Analyzer
print("")
print("    -> MULTIPERIOD")
print("")
print("        Percentiles Benchmark:")
print("")
riferimentiPercentili = [.5, .6, .7, .8, .9]
for i in riferimentiPercentili:
    numberofMultiperiodPowerfulTrends = cluster[cluster.DeltaClosePerc >= cluster["DeltaClosePerc"].quantile(i)][
        "DeltaClosePerc"].count()
    MultiperiodPowerfulTrendsPerc = round(numberofMultiperiodPowerfulTrends / len(cluster) * 100, 2)
    if cluster["DeltaClosePerc"].quantile(i) >= 0:
        print("        Percentuale di Multiperiod Trend superiori o uguali al",
              i * 100, "° percentile ({} %) ->".format(round(cluster["DeltaClosePerc"].quantile(i), 2)),
              MultiperiodPowerfulTrendsPerc, "%")

print("")
print("        Static values Benchmark:")
print("")
riferimenti = [0, 0.5, 1., 1.5, 2.]
for i in riferimenti:
    numberofMultiperiodPowerfulTrends = cluster[cluster.DeltaClosePerc >= i]["DeltaClosePerc"].count()
    MultiperiodPowerfulTrendsPerc = round(numberofMultiperiodPowerfulTrends / len(cluster) * 100, 2)
    print("        Percentuale di Multiperiod Trend superiori o uguali a", i, "%", "->", MultiperiodPowerfulTrendsPerc, "%")

print("")
print("-> DOWNTRENDS STATISTICS:")

# Intraperiod Analyzer
print("")
print("    -> INTRAPERIOD")
print("")
print("        Percentiles Benchmark:")
print("")
riferimentiPercentili = [.5, .4, .3, .2, .1]
for i in riferimentiPercentili:
    numberofIntraperiodPowerfulTrends = cluster[cluster.CloseOpenPerc <= cluster["CloseOpenPerc"].quantile(i)][
        "CloseOpenPerc"].count()
    IntraperiodPowerfulTrendsPerc = round(numberofIntraperiodPowerfulTrends / len(cluster) * 100, 2)
    if cluster["CloseOpenPerc"].quantile(i) <= 0:
        print("        Percentuale di Intraperiod Trend inferiori o uguali al",
              i * 100, "° percentile ({} %) ->".format(round(cluster["CloseOpenPerc"].quantile(i), 2)),
              IntraperiodPowerfulTrendsPerc, "%")

print("")
print("        Static values Benchmark:")
print("")
riferimenti = [0, -0.5, -1., -1.5, -2.]
for i in riferimenti:
    numberofIntraperiodPowerfulTrends = cluster[cluster.CloseOpenPerc <= i]["CloseOpenPerc"].count()
    IntraperiodPowerfulTrendsPerc = round(numberofIntraperiodPowerfulTrends / len(cluster) * 100, 2)
    print("        Percentuale di Intraperiod Trend inferiori o uguali a", i, "%", "->", IntraperiodPowerfulTrendsPerc, "%")

# Multiperiod Analyzer
print("")
print("    -> MULTIPERIOD")
print("")
print("        Percentiles Benchmark:")
print("")
riferimentiPercentili = [.5, .4, .3, .2, .1]
for i in riferimentiPercentili:
    numberofMultiperiodPowerfulTrends = cluster[cluster.DeltaClosePerc <= cluster["DeltaClosePerc"].quantile(i)][
        "DeltaClosePerc"].count()
    MultiperiodPowerfulTrendsPerc = round(numberofMultiperiodPowerfulTrends / len(cluster) * 100, 2)
    if cluster["DeltaClosePerc"].quantile(i) <= 0:
        print("        Percentuale di Multiperiod Trend inferiori o uguali al",
              i * 100, "° percentile ({} %) ->".format(round(cluster["DeltaClosePerc"].quantile(i), 2)),
              MultiperiodPowerfulTrendsPerc, "%")

print("")
print("        Static values Benchmark:")
print("")
riferimenti = [0, -0.5, -1., -1.5, -2.]
for i in riferimenti:
    numberofMultiperiodPowerfulTrends = cluster[cluster.DeltaClosePerc <= i]["DeltaClosePerc"].count()
    MultiperiodPowerfulTrendsPerc = round(numberofMultiperiodPowerfulTrends / len(cluster) * 100, 2)
    print("        Percentuale di Multiperiod Trend inferiori o uguali a", i, "%", "->", MultiperiodPowerfulTrendsPerc, "%")


GANDALF TREND ANALYZER - Powered by Gandalf Project R&D

Correlation Matrix:

                DeltaClosePerc  CloseOpenPerc
DeltaClosePerc        1.000000       0.996848
CloseOpenPerc         0.996848       1.000000

Percentiles Plot:

       DeltaClosePerc  CloseOpenPerc
count     1743.000000    1744.000000
mean        -0.017464      -0.018704
std          0.810608       0.806352
min         -6.240000      -6.230000
10%         -0.928000      -0.920000
20%         -0.550000      -0.540000
30%         -0.310000      -0.310000
40%         -0.150000      -0.140000
50%         -0.010000      -0.020000
60%          0.130000       0.118000
70%          0.290000       0.280000
80%          0.490000       0.490000
90%          0.888000       0.870000
95%          1.269000       1.258500
99%          2.271600       2.260000
max          4.830000       4.830000

-> UPTRENDS STATISTICS:

    -> INTRAPERIOD

        Percentiles Benchmark:

        Percentuale di Intraperiod Trend superiori o ugu

I valori Intraday e Multiday sono molto simili a causa dell'interruzione di solo un'ora tra le 17 e le 18 sulla sessione oraria!

### Uniamo il codice, portiamo in input la periodocità:

In [3]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__) # necessaria versione >= 1.9.0

import cufflinks as cf

# Per utilizzo con Notebooks
init_notebook_mode(connected=True)

# Per utilizzo offline
cf.go_offline()


filename = "@GC_60min.txt"
startDate = "20130101"
endDate = "20180810"
period = "D"

startDateParsed = datetime.date(int(startDate[0:4]), int(startDate[4:6]), int(startDate[6:]))
endDateParsed = datetime.date(int(endDate[0:4]), int(endDate[4:6]), int(endDate[6:]))

instrument = pd.read_csv(filename,
                         usecols=['Date','Time','Open','High','Low','Close','Up','Down'], 
                         parse_dates=[['Date', 'Time']])
print("Il file contiene", len(instrument), "record")
instrument.index = instrument['Date_Time']
instrument.drop('Date_Time',axis=1,inplace=True)
instrument['AvgPrice'] = round(instrument.iloc[:,0:4].mean(axis=1),2)
instrument['Range'] = round(instrument['High'] - instrument['Low'],2)
instrument['Body'] = abs(instrument['Open'] - instrument['Close'])
instrument['CloseOpenPerc'] = round((instrument['Close'] - instrument['Open']) / instrument['Open'] * 100,2)
instrument['BodyRangePerc'] = round(instrument['Body']/instrument['Range'] * 100,2)
# Percentuale della barra (all'interno del range) dove si attesta il close
instrument['CloseBarPerc'] = round((instrument['Close'] - instrument['Low']) * 100 / instrument['Range'],2)
instrument['DeltaClosePerc'] = round((instrument['Close'] - instrument['Close'].shift(1)) / instrument['Close'].shift(1) * 100,2)

instrument = instrument.loc[startDateParsed:endDateParsed]
#instrument.head(10)

# creazione Pandas DataFrame di lavoro con i soli campi utili
cluster = pd.DataFrame(index=instrument.Open.resample(period).first().dropna().index.copy())
cluster["Open"] = instrument.Open.resample(period).first().dropna()
cluster["Close"] = instrument.Close.resample(period).last().dropna()
cluster['DeltaClosePerc'] = round((cluster['Close'] - cluster['Close'].shift(1)) / cluster['Close'].shift(1) * 100,2)
cluster['CloseOpenPerc'] = round((cluster['Close'] - cluster['Open']) / cluster['Open'] * 100,2)
cluster.drop("Open", axis=1, inplace=True)
cluster.drop("Close", axis=1, inplace=True)

print("")
# Grafichiamo la serie di variazioni percentuali
cluster['DeltaClosePerc'].iplot(kind='bar', color="blue", title="Multiperiod Trends")
cluster['CloseOpenPerc'].iplot(kind='bar', color="red", title="Intraperiod Trends")

# Stampa Plotly&Cufflinks delle distribuzioni Intraperiod e multiperiod
cluster['DeltaClosePerc'].iplot(kind='hist',
                                bins=300,
                                color="blue",
                                title="Distribution of Multiperiod Trends")

cluster['CloseOpenPerc'].iplot(kind='hist',
                               bins=300,
                               color="red",
                               title="Distribution of Intraperiod Trends")

cluster.iplot(kind='box', color=["red", "blue"], title="Distribution of Multiperiod & Intraperiod Trends")

print("")
print("GANDALF TREND ANALYZER - Powered by Gandalf Project R&D")
print("")
print("Correlation Matrix:")
print("")
print(cluster.corr())
print("")
print("Percentiles Plot:")
print("")
print(cluster.describe(percentiles=[.1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]))

print("")
print("-> UPTRENDS STATISTICS:")

# Intraperiod Analyzer
print("")
print("    -> INTRAPERIOD")
print("")
print("        Percentiles Benchmark:")
print("")
riferimentiPercentili = [.5, .6, .7, .8, .9]
for i in riferimentiPercentili:
    numberofIntraperiodPowerfulTrends = cluster[cluster.CloseOpenPerc >= cluster["CloseOpenPerc"].quantile(i)][
        "CloseOpenPerc"].count()
    IntraperiodPowerfulTrendsPerc = round(numberofIntraperiodPowerfulTrends / len(cluster) * 100, 2)
    if cluster["CloseOpenPerc"].quantile(i) >= 0:
        print("        Percentuale di Intraperiod Trend superiori o uguali al",
              i * 100, "° percentile ({} %) ->".format(round(cluster["CloseOpenPerc"].quantile(i), 2)),
              IntraperiodPowerfulTrendsPerc, "%")

print("")
print("        Static values Benchmark:")
print("")
riferimenti = [0, 0.5, 1., 1.5, 2.]
for i in riferimenti:
    numberofIntraperiodPowerfulTrends = cluster[cluster.CloseOpenPerc >= i]["CloseOpenPerc"].count()
    IntraperiodPowerfulTrendsPerc = round(numberofIntraperiodPowerfulTrends / len(cluster) * 100, 2)
    print("        Percentuale di Intraperiod Trend superiori o uguali a", i, "%", "->", IntraperiodPowerfulTrendsPerc, "%")

# Multiperiod Analyzer
print("")
print("    -> MULTIPERIOD")
print("")
print("        Percentiles Benchmark:")
print("")
riferimentiPercentili = [.5, .6, .7, .8, .9]
for i in riferimentiPercentili:
    numberofMultiperiodPowerfulTrends = cluster[cluster.DeltaClosePerc >= cluster["DeltaClosePerc"].quantile(i)][
        "DeltaClosePerc"].count()
    MultiperiodPowerfulTrendsPerc = round(numberofMultiperiodPowerfulTrends / len(cluster) * 100, 2)
    if cluster["DeltaClosePerc"].quantile(i) >= 0:
        print("        Percentuale di Multiperiod Trend superiori o uguali al",
              i * 100, "° percentile ({} %) ->".format(round(cluster["DeltaClosePerc"].quantile(i), 2)),
              MultiperiodPowerfulTrendsPerc, "%")

print("")
print("        Static values Benchmark:")
print("")
riferimenti = [0, 0.5, 1., 1.5, 2.]
for i in riferimenti:
    numberofMultiperiodPowerfulTrends = cluster[cluster.DeltaClosePerc >= i]["DeltaClosePerc"].count()
    MultiperiodPowerfulTrendsPerc = round(numberofMultiperiodPowerfulTrends / len(cluster) * 100, 2)
    print("        Percentuale di Multiperiod Trend superiori o uguali a", i, "%", "->", MultiperiodPowerfulTrendsPerc, "%")

print("")
print("-> DOWNTRENDS STATISTICS:")

# Intraperiod Analyzer
print("")
print("    -> INTRAPERIOD")
print("")
print("        Percentiles Benchmark:")
print("")
riferimentiPercentili = [.5, .4, .3, .2, .1]
for i in riferimentiPercentili:
    numberofIntraperiodPowerfulTrends = cluster[cluster.CloseOpenPerc <= cluster["CloseOpenPerc"].quantile(i)][
        "CloseOpenPerc"].count()
    IntraperiodPowerfulTrendsPerc = round(numberofIntraperiodPowerfulTrends / len(cluster) * 100, 2)
    if cluster["CloseOpenPerc"].quantile(i) <= 0:
        print("        Percentuale di Intraperiod Trend inferiori o uguali al",
              i * 100, "° percentile ({} %) ->".format(round(cluster["CloseOpenPerc"].quantile(i), 2)),
              IntraperiodPowerfulTrendsPerc, "%")

print("")
print("        Static values Benchmark:")
print("")
riferimenti = [0, -0.5, -1., -1.5, -2.]
for i in riferimenti:
    numberofIntraperiodPowerfulTrends = cluster[cluster.CloseOpenPerc <= i]["CloseOpenPerc"].count()
    IntraperiodPowerfulTrendsPerc = round(numberofIntraperiodPowerfulTrends / len(cluster) * 100, 2)
    print("        Percentuale di Intraperiod Trend inferiori o uguali a", i, "%", "->", IntraperiodPowerfulTrendsPerc, "%")

# Multiperiod Analyzer
print("")
print("    -> MULTIPERIOD")
print("")
print("        Percentiles Benchmark:")
print("")
riferimentiPercentili = [.5, .4, .3, .2, .1]
for i in riferimentiPercentili:
    numberofMultiperiodPowerfulTrends = cluster[cluster.DeltaClosePerc <= cluster["DeltaClosePerc"].quantile(i)][
        "DeltaClosePerc"].count()
    MultiperiodPowerfulTrendsPerc = round(numberofMultiperiodPowerfulTrends / len(cluster) * 100, 2)
    if cluster["DeltaClosePerc"].quantile(i) <= 0:
        print("        Percentuale di Multiperiod Trend inferiori o uguali al",
              i * 100, "° percentile ({} %) ->".format(round(cluster["DeltaClosePerc"].quantile(i), 2)),
              MultiperiodPowerfulTrendsPerc, "%")

print("")
print("        Static values Benchmark:")
print("")
riferimenti = [0, -0.5, -1., -1.5, -2.]
for i in riferimenti:
    numberofMultiperiodPowerfulTrends = cluster[cluster.DeltaClosePerc <= i]["DeltaClosePerc"].count()
    MultiperiodPowerfulTrendsPerc = round(numberofMultiperiodPowerfulTrends / len(cluster) * 100, 2)
    print("        Percentuale di Multiperiod Trend inferiori o uguali a", i, "%", "->", MultiperiodPowerfulTrendsPerc, "%")

4.7.1


Il file contiene 68540 record




GANDALF TREND ANALYZER - Powered by Gandalf Project R&D

Correlation Matrix:

                DeltaClosePerc  CloseOpenPerc
DeltaClosePerc        1.000000       0.996848
CloseOpenPerc         0.996848       1.000000

Percentiles Plot:

       DeltaClosePerc  CloseOpenPerc
count     1743.000000    1744.000000
mean        -0.017464      -0.018704
std          0.810608       0.806352
min         -6.240000      -6.230000
10%         -0.928000      -0.920000
20%         -0.550000      -0.540000
30%         -0.310000      -0.310000
40%         -0.150000      -0.140000
50%         -0.010000      -0.020000
60%          0.130000       0.118000
70%          0.290000       0.280000
80%          0.490000       0.490000
90%          0.888000       0.870000
95%          1.269000       1.258500
99%          2.271600       2.260000
max          4.830000       4.830000

-> UPTRENDS STATISTICS:

    -> INTRAPERIOD

        Percentiles Benchmark:

        Percentuale di Intraperiod Trend superiori o ugu

## Operiamo adesso su base settimanale (Weekly):

In [4]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__) # necessaria versione >= 1.9.0

import cufflinks as cf

# Per utilizzo con Notebooks
init_notebook_mode(connected=True)

# Per utilizzo offline
cf.go_offline()


filename = "@GC_60min.txt"
startDate = "20130101"
endDate = "20180810"
period = "W"

startDateParsed = datetime.date(int(startDate[0:4]), int(startDate[4:6]), int(startDate[6:]))
endDateParsed = datetime.date(int(endDate[0:4]), int(endDate[4:6]), int(endDate[6:]))

instrument = pd.read_csv(filename,
                         usecols=['Date','Time','Open','High','Low','Close','Up','Down'], 
                         parse_dates=[['Date', 'Time']])
print("Il file contiene", len(instrument), "record")
instrument.index = instrument['Date_Time']
instrument.drop('Date_Time',axis=1,inplace=True)
instrument['AvgPrice'] = round(instrument.iloc[:,0:4].mean(axis=1),2)
instrument['Range'] = round(instrument['High'] - instrument['Low'],2)
instrument['Body'] = abs(instrument['Open'] - instrument['Close'])
instrument['CloseOpenPerc'] = round((instrument['Close'] - instrument['Open']) / instrument['Open'] * 100,2)
instrument['BodyRangePerc'] = round(instrument['Body']/instrument['Range'] * 100,2)
# Percentuale della barra (all'interno del range) dove si attesta il close
instrument['CloseBarPerc'] = round((instrument['Close'] - instrument['Low']) * 100 / instrument['Range'],2)
instrument['DeltaClosePerc'] = round((instrument['Close'] - instrument['Close'].shift(1)) / instrument['Close'].shift(1) * 100,2)

instrument = instrument.loc[startDateParsed:endDateParsed]
#instrument.head(10)

# creazione Pandas DataFrame di lavoro con i soli campi utili
cluster = pd.DataFrame(index=instrument.Open.resample(period).first().dropna().index.copy())
cluster["Open"] = instrument.Open.resample(period).first().dropna()
cluster["Close"] = instrument.Close.resample(period).last().dropna()
cluster['DeltaClosePerc'] = round((cluster['Close'] - cluster['Close'].shift(1)) / cluster['Close'].shift(1) * 100,2)
cluster['CloseOpenPerc'] = round((cluster['Close'] - cluster['Open']) / cluster['Open'] * 100,2)
cluster.drop("Open", axis=1, inplace=True)
cluster.drop("Close", axis=1, inplace=True)

print("")
# Grafichiamo la serie di variazioni percentuali
cluster['DeltaClosePerc'].iplot(kind='bar', color="blue", title="Multiperiod Trends")
cluster['CloseOpenPerc'].iplot(kind='bar', color="red", title="Intraperiod Trends")

# Stampa Plotly&Cufflinks delle distribuzioni Intraperiod e multiperiod
cluster['DeltaClosePerc'].iplot(kind='hist',
                                bins=300,
                                color="blue",
                                title="Distribution of Multiperiod Trends")

cluster['CloseOpenPerc'].iplot(kind='hist',
                               bins=300,
                               color="red",
                               title="Distribution of Intraperiod Trends")

cluster.iplot(kind='box', color=["red", "blue"], title="Distribution of Multiperiod & Intraperiod Trends")

print("")
print("GANDALF TREND ANALYZER - Powered by Gandalf Project R&D")
print("")
print("Correlation Matrix:")
print("")
print(cluster.corr())
print("")
print("Percentiles Plot:")
print("")
print(cluster.describe(percentiles=[.1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]))

print("")
print("-> UPTRENDS STATISTICS:")

# Intraperiod Analyzer
print("")
print("    -> INTRAPERIOD")
print("")
print("        Percentiles Benchmark:")
print("")
riferimentiPercentili = [.5, .6, .7, .8, .9]
for i in riferimentiPercentili:
    numberofIntraperiodPowerfulTrends = cluster[cluster.CloseOpenPerc >= cluster["CloseOpenPerc"].quantile(i)][
        "CloseOpenPerc"].count()
    IntraperiodPowerfulTrendsPerc = round(numberofIntraperiodPowerfulTrends / len(cluster) * 100, 2)
    if cluster["CloseOpenPerc"].quantile(i) >= 0:
        print("        Percentuale di Intraperiod Trend superiori o uguali al",
              i * 100, "° percentile ({} %) ->".format(round(cluster["CloseOpenPerc"].quantile(i), 2)),
              IntraperiodPowerfulTrendsPerc, "%")

print("")
print("        Static values Benchmark:")
print("")
riferimenti = [0, 0.5, 1., 1.5, 2.]
for i in riferimenti:
    numberofIntraperiodPowerfulTrends = cluster[cluster.CloseOpenPerc >= i]["CloseOpenPerc"].count()
    IntraperiodPowerfulTrendsPerc = round(numberofIntraperiodPowerfulTrends / len(cluster) * 100, 2)
    print("        Percentuale di Intraperiod Trend superiori o uguali a", i, "%", "->", IntraperiodPowerfulTrendsPerc, "%")

# Multiperiod Analyzer
print("")
print("    -> MULTIPERIOD")
print("")
print("        Percentiles Benchmark:")
print("")
riferimentiPercentili = [.5, .6, .7, .8, .9]
for i in riferimentiPercentili:
    numberofMultiperiodPowerfulTrends = cluster[cluster.DeltaClosePerc >= cluster["DeltaClosePerc"].quantile(i)][
        "DeltaClosePerc"].count()
    MultiperiodPowerfulTrendsPerc = round(numberofMultiperiodPowerfulTrends / len(cluster) * 100, 2)
    if cluster["DeltaClosePerc"].quantile(i) >= 0:
        print("        Percentuale di Multiperiod Trend superiori o uguali al",
              i * 100, "° percentile ({} %) ->".format(round(cluster["DeltaClosePerc"].quantile(i), 2)),
              MultiperiodPowerfulTrendsPerc, "%")

print("")
print("        Static values Benchmark:")
print("")
riferimenti = [0, 0.5, 1., 1.5, 2.]
for i in riferimenti:
    numberofMultiperiodPowerfulTrends = cluster[cluster.DeltaClosePerc >= i]["DeltaClosePerc"].count()
    MultiperiodPowerfulTrendsPerc = round(numberofMultiperiodPowerfulTrends / len(cluster) * 100, 2)
    print("        Percentuale di Multiperiod Trend superiori o uguali a", i, "%", "->", MultiperiodPowerfulTrendsPerc, "%")

print("")
print("-> DOWNTRENDS STATISTICS:")

# Intraperiod Analyzer
print("")
print("    -> INTRAPERIOD")
print("")
print("        Percentiles Benchmark:")
print("")
riferimentiPercentili = [.5, .4, .3, .2, .1]
for i in riferimentiPercentili:
    numberofIntraperiodPowerfulTrends = cluster[cluster.CloseOpenPerc <= cluster["CloseOpenPerc"].quantile(i)][
        "CloseOpenPerc"].count()
    IntraperiodPowerfulTrendsPerc = round(numberofIntraperiodPowerfulTrends / len(cluster) * 100, 2)
    if cluster["CloseOpenPerc"].quantile(i) <= 0:
        print("        Percentuale di Intraperiod Trend inferiori o uguali al",
              i * 100, "° percentile ({} %) ->".format(round(cluster["CloseOpenPerc"].quantile(i), 2)),
              IntraperiodPowerfulTrendsPerc, "%")

print("")
print("        Static values Benchmark:")
print("")
riferimenti = [0, -0.5, -1., -1.5, -2.]
for i in riferimenti:
    numberofIntraperiodPowerfulTrends = cluster[cluster.CloseOpenPerc <= i]["CloseOpenPerc"].count()
    IntraperiodPowerfulTrendsPerc = round(numberofIntraperiodPowerfulTrends / len(cluster) * 100, 2)
    print("        Percentuale di Intraperiod Trend inferiori o uguali a", i, "%", "->", IntraperiodPowerfulTrendsPerc, "%")

# Multiperiod Analyzer
print("")
print("    -> MULTIPERIOD")
print("")
print("        Percentiles Benchmark:")
print("")
riferimentiPercentili = [.5, .4, .3, .2, .1]
for i in riferimentiPercentili:
    numberofMultiperiodPowerfulTrends = cluster[cluster.DeltaClosePerc <= cluster["DeltaClosePerc"].quantile(i)][
        "DeltaClosePerc"].count()
    MultiperiodPowerfulTrendsPerc = round(numberofMultiperiodPowerfulTrends / len(cluster) * 100, 2)
    if cluster["DeltaClosePerc"].quantile(i) <= 0:
        print("        Percentuale di Multiperiod Trend inferiori o uguali al",
              i * 100, "° percentile ({} %) ->".format(round(cluster["DeltaClosePerc"].quantile(i), 2)),
              MultiperiodPowerfulTrendsPerc, "%")

print("")
print("        Static values Benchmark:")
print("")
riferimenti = [0, -0.5, -1., -1.5, -2.]
for i in riferimenti:
    numberofMultiperiodPowerfulTrends = cluster[cluster.DeltaClosePerc <= i]["DeltaClosePerc"].count()
    MultiperiodPowerfulTrendsPerc = round(numberofMultiperiodPowerfulTrends / len(cluster) * 100, 2)
    print("        Percentuale di Multiperiod Trend inferiori o uguali a", i, "%", "->", MultiperiodPowerfulTrendsPerc, "%")

4.7.1


Il file contiene 68540 record




GANDALF TREND ANALYZER - Powered by Gandalf Project R&D

Correlation Matrix:

                DeltaClosePerc  CloseOpenPerc
DeltaClosePerc        1.000000       0.999991
CloseOpenPerc         0.999991       1.000000

Percentiles Plot:

       DeltaClosePerc  CloseOpenPerc
count      292.000000     293.000000
mean        -0.100205      -0.102287
std          1.984981       1.982412
min         -8.330000      -8.330000
10%         -2.318000      -2.316000
20%         -1.480000      -1.480000
30%         -1.027000      -1.024000
40%         -0.600000      -0.616000
50%         -0.155000      -0.160000
60%          0.410000       0.410000
70%          0.901000       0.892000
80%          1.444000       1.444000
90%          2.338000       2.330000
95%          3.124000       3.122000
99%          4.287100       4.266800
max          5.500000       5.490000

-> UPTRENDS STATISTICS:

    -> INTRAPERIOD

        Percentiles Benchmark:

        Percentuale di Intraperiod Trend superiori o ugu

## Ed infine su base mensile (Monthly):

In [5]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__) # necessaria versione >= 1.9.0

import cufflinks as cf

# Per utilizzo con Notebooks
init_notebook_mode(connected=True)

# Per utilizzo offline
cf.go_offline()


filename = "@GC_60min.txt"
startDate = "20130101"
endDate = "20180810"
period = "M"

startDateParsed = datetime.date(int(startDate[0:4]), int(startDate[4:6]), int(startDate[6:]))
endDateParsed = datetime.date(int(endDate[0:4]), int(endDate[4:6]), int(endDate[6:]))

instrument = pd.read_csv(filename,
                         usecols=['Date','Time','Open','High','Low','Close','Up','Down'], 
                         parse_dates=[['Date', 'Time']])
print("Il file contiene", len(instrument), "record")
instrument.index = instrument['Date_Time']
instrument.drop('Date_Time',axis=1,inplace=True)
instrument['AvgPrice'] = round(instrument.iloc[:,0:4].mean(axis=1),2)
instrument['Range'] = round(instrument['High'] - instrument['Low'],2)
instrument['Body'] = abs(instrument['Open'] - instrument['Close'])
instrument['CloseOpenPerc'] = round((instrument['Close'] - instrument['Open']) / instrument['Open'] * 100,2)
instrument['BodyRangePerc'] = round(instrument['Body']/instrument['Range'] * 100,2)
# Percentuale della barra (all'interno del range) dove si attesta il close
instrument['CloseBarPerc'] = round((instrument['Close'] - instrument['Low']) * 100 / instrument['Range'],2)
instrument['DeltaClosePerc'] = round((instrument['Close'] - instrument['Close'].shift(1)) / instrument['Close'].shift(1) * 100,2)

instrument = instrument.loc[startDateParsed:endDateParsed]
#instrument.head(10)

# creazione Pandas DataFrame di lavoro con i soli campi utili
cluster = pd.DataFrame(index=instrument.Open.resample(period).first().dropna().index.copy())
cluster["Open"] = instrument.Open.resample(period).first().dropna()
cluster["Close"] = instrument.Close.resample(period).last().dropna()
cluster['DeltaClosePerc'] = round((cluster['Close'] - cluster['Close'].shift(1)) / cluster['Close'].shift(1) * 100,2)
cluster['CloseOpenPerc'] = round((cluster['Close'] - cluster['Open']) / cluster['Open'] * 100,2)
cluster.drop("Open", axis=1, inplace=True)
cluster.drop("Close", axis=1, inplace=True)

print("")
# Grafichiamo la serie di variazioni percentuali
cluster['DeltaClosePerc'].iplot(kind='bar', color="blue", title="Multiperiod Trends")
cluster['CloseOpenPerc'].iplot(kind='bar', color="red", title="Intraperiod Trends")

# Stampa Plotly&Cufflinks delle distribuzioni Intraperiod e multiperiod
cluster['DeltaClosePerc'].iplot(kind='hist',
                                bins=300,
                                color="blue",
                                title="Distribution of Multiperiod Trends")

cluster['CloseOpenPerc'].iplot(kind='hist',
                               bins=300,
                               color="red",
                               title="Distribution of Intraperiod Trends")

cluster.iplot(kind='box', color=["red", "blue"], title="Distribution of Multiperiod & Intraperiod Trends")

print("")
print("GANDALF TREND ANALYZER - Powered by Gandalf Project R&D")
print("")
print("Correlation Matrix:")
print("")
print(cluster.corr())
print("")
print("Percentiles Plot:")
print("")
print(cluster.describe(percentiles=[.1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]))

print("")
print("-> UPTRENDS STATISTICS:")

# Intraperiod Analyzer
print("")
print("    -> INTRAPERIOD")
print("")
print("        Percentiles Benchmark:")
print("")
riferimentiPercentili = [.5, .6, .7, .8, .9]
for i in riferimentiPercentili:
    numberofIntraperiodPowerfulTrends = cluster[cluster.CloseOpenPerc >= cluster["CloseOpenPerc"].quantile(i)][
        "CloseOpenPerc"].count()
    IntraperiodPowerfulTrendsPerc = round(numberofIntraperiodPowerfulTrends / len(cluster) * 100, 2)
    if cluster["CloseOpenPerc"].quantile(i) >= 0:
        print("        Percentuale di Intraperiod Trend superiori o uguali al",
              i * 100, "° percentile ({} %) ->".format(round(cluster["CloseOpenPerc"].quantile(i), 2)),
              IntraperiodPowerfulTrendsPerc, "%")

print("")
print("        Static values Benchmark:")
print("")
riferimenti = [0, 0.5, 1., 1.5, 2.]
for i in riferimenti:
    numberofIntraperiodPowerfulTrends = cluster[cluster.CloseOpenPerc >= i]["CloseOpenPerc"].count()
    IntraperiodPowerfulTrendsPerc = round(numberofIntraperiodPowerfulTrends / len(cluster) * 100, 2)
    print("        Percentuale di Intraperiod Trend superiori o uguali a", i, "%", "->", IntraperiodPowerfulTrendsPerc, "%")

# Multiperiod Analyzer
print("")
print("    -> MULTIPERIOD")
print("")
print("        Percentiles Benchmark:")
print("")
riferimentiPercentili = [.5, .6, .7, .8, .9]
for i in riferimentiPercentili:
    numberofMultiperiodPowerfulTrends = cluster[cluster.DeltaClosePerc >= cluster["DeltaClosePerc"].quantile(i)][
        "DeltaClosePerc"].count()
    MultiperiodPowerfulTrendsPerc = round(numberofMultiperiodPowerfulTrends / len(cluster) * 100, 2)
    if cluster["DeltaClosePerc"].quantile(i) >= 0:
        print("        Percentuale di Multiperiod Trend superiori o uguali al",
              i * 100, "° percentile ({} %) ->".format(round(cluster["DeltaClosePerc"].quantile(i), 2)),
              MultiperiodPowerfulTrendsPerc, "%")

print("")
print("        Static values Benchmark:")
print("")
riferimenti = [0, 0.5, 1., 1.5, 2.]
for i in riferimenti:
    numberofMultiperiodPowerfulTrends = cluster[cluster.DeltaClosePerc >= i]["DeltaClosePerc"].count()
    MultiperiodPowerfulTrendsPerc = round(numberofMultiperiodPowerfulTrends / len(cluster) * 100, 2)
    print("        Percentuale di Multiperiod Trend superiori o uguali a", i, "%", "->", MultiperiodPowerfulTrendsPerc, "%")

print("")
print("-> DOWNTRENDS STATISTICS:")

# Intraperiod Analyzer
print("")
print("    -> INTRAPERIOD")
print("")
print("        Percentiles Benchmark:")
print("")
riferimentiPercentili = [.5, .4, .3, .2, .1]
for i in riferimentiPercentili:
    numberofIntraperiodPowerfulTrends = cluster[cluster.CloseOpenPerc <= cluster["CloseOpenPerc"].quantile(i)][
        "CloseOpenPerc"].count()
    IntraperiodPowerfulTrendsPerc = round(numberofIntraperiodPowerfulTrends / len(cluster) * 100, 2)
    if cluster["CloseOpenPerc"].quantile(i) <= 0:
        print("        Percentuale di Intraperiod Trend inferiori o uguali al",
              i * 100, "° percentile ({} %) ->".format(round(cluster["CloseOpenPerc"].quantile(i), 2)),
              IntraperiodPowerfulTrendsPerc, "%")

print("")
print("        Static values Benchmark:")
print("")
riferimenti = [0, -0.5, -1., -1.5, -2.]
for i in riferimenti:
    numberofIntraperiodPowerfulTrends = cluster[cluster.CloseOpenPerc <= i]["CloseOpenPerc"].count()
    IntraperiodPowerfulTrendsPerc = round(numberofIntraperiodPowerfulTrends / len(cluster) * 100, 2)
    print("        Percentuale di Intraperiod Trend inferiori o uguali a", i, "%", "->", IntraperiodPowerfulTrendsPerc, "%")

# Multiperiod Analyzer
print("")
print("    -> MULTIPERIOD")
print("")
print("        Percentiles Benchmark:")
print("")
riferimentiPercentili = [.5, .4, .3, .2, .1]
for i in riferimentiPercentili:
    numberofMultiperiodPowerfulTrends = cluster[cluster.DeltaClosePerc <= cluster["DeltaClosePerc"].quantile(i)][
        "DeltaClosePerc"].count()
    MultiperiodPowerfulTrendsPerc = round(numberofMultiperiodPowerfulTrends / len(cluster) * 100, 2)
    if cluster["DeltaClosePerc"].quantile(i) <= 0:
        print("        Percentuale di Multiperiod Trend inferiori o uguali al",
              i * 100, "° percentile ({} %) ->".format(round(cluster["DeltaClosePerc"].quantile(i), 2)),
              MultiperiodPowerfulTrendsPerc, "%")

print("")
print("        Static values Benchmark:")
print("")
riferimenti = [0, -0.5, -1., -1.5, -2.]
for i in riferimenti:
    numberofMultiperiodPowerfulTrends = cluster[cluster.DeltaClosePerc <= i]["DeltaClosePerc"].count()
    MultiperiodPowerfulTrendsPerc = round(numberofMultiperiodPowerfulTrends / len(cluster) * 100, 2)
    print("        Percentuale di Multiperiod Trend inferiori o uguali a", i, "%", "->", MultiperiodPowerfulTrendsPerc, "%")

4.7.1


Il file contiene 68540 record




GANDALF TREND ANALYZER - Powered by Gandalf Project R&D

Correlation Matrix:

                DeltaClosePerc  CloseOpenPerc
DeltaClosePerc        1.000000       0.999882
CloseOpenPerc         0.999882       1.000000

Percentiles Plot:

       DeltaClosePerc  CloseOpenPerc
count       67.000000      68.000000
mean        -0.435672      -0.442794
std          4.208747       4.183487
min        -10.180000     -10.270000
10%         -5.736000      -5.671000
20%         -3.498000      -3.468000
30%         -2.742000      -2.710000
40%         -1.734000      -1.624000
50%         -0.930000      -0.920000
60%          0.346000       0.302000
70%          1.266000       1.236000
80%          2.764000       2.752000
90%          5.368000       5.323000
95%          6.304000       6.429500
99%          9.287000       9.284800
max         10.310000      10.330000

-> UPTRENDS STATISTICS:

    -> INTRAPERIOD

        Percentiles Benchmark:

        Percentuale di Intraperiod Trend superiori o ugu

### La conclusione è che, viste le basse percentuali in trend accorciando la periodicità, abbiamo bisogno di filtrare i trade trend following se vogliamo erodere valore (anche in intradaperiod)!